Your FIFA project goes here:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./file_for_project/fifa21_training.csv")

#Consulta de valores NaN:
#for elem in data.columns:
#    print(data[elem].isna().sum(),elem)

In [3]:
data.columns = [e.lower().replace(' ' , '_') for e in data.columns] #unificamos formato nombres

data = data.drop(columns=["unnamed:_0","id",'name','bp','nationality','team_&_contract'
        ,'club','joined','loan_date_end','contract'])

data['position'] = data['position'].fillna('0')

data = data.dropna()

columns = data.columns.tolist()

# A position la convertiremos en número de posiciones en las que puede jugar
# Qué es growth?
# Club, convertir a si es de gran liga o otro
# Wage, release_clause --> Quitar el símbolo de moneda, y cambiar la K y la M por los correspondientes 0.
# volleys, curve, agility, jumping, balance, interceptions, positioning, vision, composure, sliding_tackle  --> Son con decimales. Lo podemos dejar?
# w/f, sm --> Quitar el espacio y la estrella. Quedarnos sólo el indice [0]
# a/w, d/w, ir --> convertir High/Medium/Low en 1,2,3
# hits --> Quitar las comillas de string y K y M --> Se ha metido en la función dinero
# a partir de ls y hasta gk, mirar qué hacer con los símbolos +

In [4]:
#Funciones

def dinero(x):
    if '.' in x and 'K' in x:
        x=x.replace('K','00')
        x=x.replace('.','')
    else:
        x=x.replace('K','000')
    
    if '.' in x and 'M' in x:
        x=x.replace('M','00000')
        x=x.replace('.','')
    else:
        x=x.replace('M','000000')
        
    x=x.replace('€','')
    x=x.replace('$','')
    x=int(x)
    return x

def estrella(x):
    return int(x[0])

def himelo(x):
    if x == 'High':
        return 3
    if x == 'Medium':
        return 2
    if x == 'Low':
        return 1
    
def peso(x):
    return int(x.split('l')[0])

def altura(x):
    metros = int(x[0])*0.3048*100
    centimetros = int(x[-2])*2.54
    return metros+centimetros

In [5]:
# Conversión de columnas relativamente directa

data['position'] = [len(a.split(' ')) if a!='0' else 0 for a in data['position']]
data['wage'] = [dinero(wage) for wage in data['wage']]
data['release_clause'] = [dinero(rc) for rc in data['release_clause']]
data['value'] = [dinero(value) for value in data['value']]
data['w/f'] = [estrella(x) for x in data['w/f']]
data['sm'] = [estrella(x) for x in data['sm']]
data['a/w'] = [himelo(x) for x in data['a/w']]
data['d/w'] = [himelo(x) for x in data['d/w']]
data['ir'] = [estrella(x) for x in data['ir']]
data['hits'] = [dinero(x) for x in data['hits']]
data['weight'] = [peso(w) for w in data['weight']]
data['height'] = [altura(h) for h in data['height']]

In [6]:
# Conversión en masa de algunas columnas

col_con_mas = ['ls','st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']

for i in col_con_mas:
    data[i+'+'] = [int(x.split('+')[1]) for x in data[i]]
    data[i] = [int(x.split('+')[0]) for x in data[i]]

In [7]:
# Estudio de la información

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Matriz de correlaciones

#corr = data.corr()
#mask = np.triu(np.ones_like(corr, dtype=np.bool))
#f, ax = plt.subplots(figsize=(11, 9))
#cmap = sns.diverging_palette(220, 10, as_cmap=True)

#sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={'shrink':.5});

# Salvo para las variables de portero, no se aprecian correlaciones significativas

In [8]:
# Estandarizar columnas numéricas (Todas entre -1 y 1) --> Quizá es mejor quitando antes outliers

from sklearn.preprocessing import StandardScaler

num_cols = [col for col in data.columns if (data[col].dtype!='object')]

#for col in num_cols[:-1]:
 #   data[col] = StandardScaler().fit_transform(data[col].values.reshape(-1,1))

In [9]:
#for c in num_cols:
 #   plt.figure(figsize=(10,5))
  #  plt.boxplot(data[c])
   # plt.title(c)
    #plt.show();
    
log_cols = ['value','wage','release_clause','goalkeeping','gk_diving'
            ,'gk_handling','gk_kicking','gk_positioning','gk_reflexes']

In [10]:
def log_transfom_clean1(x):
    x = np.log(x)
    if np.isfinite(x): #para protegernos del hecho de que log(0) = infinito
        return x
    else:
        return 0

for i in log_cols:
    data[i+'_log'] = list(map(log_transfom_clean1, data[i]))

C:\Users\EB885UN\AppData\Local\Temp\ipykernel_21188\2742457897.py:2: RuntimeWarning: divide by zero encountered in log
  x = np.log(x)
C:\Users\EB885UN\AppData\Local\Temp\ipykernel_21188\2742457897.py:2: RuntimeWarning: divide by zero encountered in log
  x = np.log(x)
C:\Users\EB885UN\AppData\Local\Temp\ipykernel_21188\2742457897.py:2: RuntimeWarning: divide by zero encountered in log
  x = np.log(x)


In [11]:
#for i in log_cols:   
#    plt.figure(figsize=(10,5))
#    plt.boxplot(data[i+'_log'])
#    plt.title(i+'_log')
#    plt.show();

In [12]:
for i in log_cols:
    data = data.loc[data[i+'_log'] > 0 ]
#    sns.distplot(data[i+'_log'])
#    plt.show()
    

for i in log_cols:
    data.drop(i, axis=1, inplace=True)

In [13]:
# One Hot Data de la información categórica

data['foot'] = pd.get_dummies(data['foot'], drop_first=True)
data['foot']

0        1
2        1
3        1
4        1
5        1
        ..
13695    0
13696    1
13697    0
13698    1
13699    1
Name: foot, Length: 12267, dtype: uint8

In [27]:
cols = list(data.columns)
cols.remove('ova')

In [16]:
#Normalizar

for c in data[cols]:   # we'll normalize all less the target column
    data[c]=StandardScaler().fit_transform(data[c].values.reshape(-1,1))

In [17]:
# first, split X-y (learning-target data)
X=data.drop(columns=['ova'])
y=data['ova']

# checking shape
print(X.shape)
print(y.shape)

# train_test_split
from sklearn.model_selection import train_test_split as tts

# train-test-split (4 sets)

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)  # random state fixed sample

(12267, 118)
(12267,)


In [18]:
# Crear los modelos

from sklearn.linear_model import LinearRegression as LinReg
linreg=LinReg()    # model
linreg.fit(X_train, y_train)   # model train
y_pred_linreg=linreg.predict(X_test)   # model prediction

from sklearn.linear_model import Lasso       # L1
from sklearn.linear_model import Ridge       # L2
from sklearn.linear_model import ElasticNet  # L1+L2

# Lasso L1

lasso=Lasso()
lasso.fit(X_train, y_train)

y_pred_lasso = lasso.predict(X_test)

# Ridge L2

ridge=Ridge()
ridge.fit(X_train, y_train)

y_pred_ridge = ridge.predict(X_test)

# ElasticNet L1+L2

elastic=ElasticNet()
elastic.fit(X_train, y_train)

y_pred_elastic = elastic.predict(X_test)

from sklearn.ensemble import RandomForestRegressor as RFR

rfr=RFR()
rfr.fit(X_train, y_train)

y_pred_rfr = rfr.predict(X_test)

from xgboost import XGBRegressor as XGBR

xgbr=XGBR()
xgbr.fit(X_train, y_train)

y_pred_xgbr = xgbr.predict(X_test)

from lightgbm import LGBMRegressor as LGBMR

lgbmr=LGBMR()
lgbmr.fit(X_train, y_train)

y_pred_lgbmr = lgbmr.predict(X_test)

models=[linreg, lasso, ridge, elastic, rfr, xgbr, lgbmr]
model_names=['linreg', 'lasso', 'ridge', 'elastic', 'rfr', 'xgbr', 'lgbmr']
preds=[y_pred_linreg, y_pred_lasso, y_pred_ridge, y_pred_elastic, y_pred_rfr, y_pred_xgbr, y_pred_lgbmr]

for i in range(len(models)):

    train_score=models[i].score(X_train, y_train) #R2
    test_score=models[i].score(X_test, y_test)

    print ('Model: {}, train R2: {} -- test R2: {}'.format(model_names[i], train_score, test_score))

    

C:\Users\EB885UN\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\EB885UN\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Model: linreg, train R2: 0.9728690297447633 -- test R2: 0.9745579247887602
Model: lasso, train R2: 0.9415315584722402 -- test R2: 0.9443437170926562
Model: ridge, train R2: 0.9823240434559808 -- test R2: 0.9835217941684088
Model: elastic, train R2: 0.9319528077806686 -- test R2: 0.9330366528972902
Model: rfr, train R2: 0.9994875986958061 -- test R2: 0.9968370765833514
Model: xgbr, train R2: 0.9995190395518598 -- test R2: 0.996547653654479
Model: lgbmr, train R2: 0.9988172904021897 -- test R2: 0.9975520021675949


In [28]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

mae = []
mse = []
rmse = []
mape = []
r2 = []

for i in preds:
    mae.append(np.round(mean_absolute_error(y_test, i), 2))
    mse.append(np.round(mean_squared_error(y_test,i),2))
    rmse.append(np.round(np.sqrt(mse[-1]),2))
    mape.append(np.round(np.mean((y_test - i)/y)*100,2))
    r2.append(np.round(r2_score(y_test, i), 2))

In [20]:
mae

[0.87, 1.21, 0.69, 1.38, 0.22, 0.26, 0.23]

In [21]:
mse

[1.24, 2.72, 0.8, 3.27, 0.15, 0.17, 0.12]

In [22]:
rmse

[1.11, 1.65, 0.89, 1.81, 0.39, 0.41, 0.35]

In [23]:
r2

[0.97, 0.94, 0.98, 0.93, 1.0, 1.0, 1.0]